<a href="https://colab.research.google.com/github/MarceloClaro/QuantumDeepClassifier/blob/main/QuantumDeepClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit
!pip install pennylane
!pip install tensorflow-quantum
!pip install matplotlib
!pip install pillow


In [ ]:
import zipfile
import os

# Caminho do arquivo zip carregado
zip_path = '/content/melanomas.zip'
extract_path = '/content/melanomas'

# Extraindo o arquivo zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Listando os arquivos extraídos
extracted_files = []
for root, dirs, files in os.walk(extract_path):
    for file in files:
        extracted_files.append(os.path.join(root, file))

extracted_files[:10]  # Mostrando os primeiros 10 arquivos para análise


In [ ]:
from PIL import Image
import numpy as np

# Caminho de amostra para imagens
sample_image_path = extracted_files[0]

# Abrindo e processando a imagem
image = Image.open(sample_image_path)
image_resized = image.resize((64, 64))  # Redimensionando para 64x64 pixels
image_array = np.array(image_resized) / 255.0  # Normalizando os valores entre 0 e 1

# Verificando as dimensões e valores da imagem processada
image_array.shape, image_array[:5, :5, 0]  # Mostrando a dimensão e parte da matriz para confirmação


In [ ]:
import glob

# Caminho das pastas para cada classe
class_dirs = [os.path.join(extract_path, "benigno")]
image_size = (64, 64)

# Função para processar imagens
def process_images(image_paths, image_size):
    images = []
    labels = []
    for image_path in image_paths:
        # Identificando a classe a partir do caminho
        label = os.path.basename(os.path.dirname(image_path))

        # Abrindo, redimensionando e normalizando a imagem
        image = Image.open(image_path).resize(image_size)
        image_array = np.array(image) / 255.0
        images.append(image_array)
        labels.append(label)
    return np.array(images), labels

# Coletando todas as imagens do diretório
all_image_paths = [img for class_dir in class_dirs for img in glob.glob(f"{class_dir}/*.jpg")]
processed_images, labels = process_images(all_image_paths, image_size)

# Mostrando o formato dos dados processados
processed_images.shape, len(labels)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Convertendo rótulos de texto para valores numéricos
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Transformando imagens em vetores unidimensionais
flattened_images = processed_images.reshape(processed_images.shape[0], -1)

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(flattened_images, encoded_labels, test_size=0.2, random_state=42)

# Mostrando as dimensões dos dados preparados
X_train.shape, X_test.shape, len(y_train), len(y_test)


In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Configuração do dispositivo quântico
n_qubits = 10  # Número de qubits usados
dev = qml.device("default.qubit", wires=n_qubits)

# Função de embedding quântico
def data_embedding(features, wires):
    """Embed dados clássicos em estados quânticos"""
    for i, wire in enumerate(wires):
        qml.RY(features[i], wires=wire)

# Função do modelo quântico
def quantum_model(weights, features):
    """Modelo quântico parametrizado"""
    data_embedding(features, range(n_qubits))  # Embedding dos dados
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))  # Medida no primeiro qubit

# QNode com Pennylane
n_layers = 3
weights_shape = (n_layers, n_qubits)
weights = np.random.random(weights_shape)  # Pesos com formato 2D

@qml.qnode(dev)
def circuit(weights, features):
    return quantum_model(weights, features)

# Exemplo de execução com os primeiros 10 elementos de uma amostra
sample_features = X_train[0][:n_qubits]  # Selecionando exatamente 10 elementos

# Certificando-se de que sample_features tem o tamanho correto
sample_features = np.pad(sample_features, (0, n_qubits - len(sample_features))) if len(sample_features) < n_qubits else sample_features[:n_qubits]

# Executando o circuito
print("Resultado do circuito:", circuit(weights, sample_features))


In [ ]:
from pennylane.optimize import AdamOptimizer

# Função de custo
def cost(weights, X, y):
    loss = 0
    for i in range(len(X)):
        pred = circuit(weights, X[i])
        loss += (pred - y[i])**2
    return loss / len(X)

# Configurando o otimizador
opt = AdamOptimizer(stepsize=0.01)
steps = 100  # Número de iterações
weights = np.random.random(weights_shape)  # Reinicializando pesos

# Treinamento
for step in range(steps):
    weights = opt.step(lambda w: cost(w, X_train[:50, :n_qubits], y_train[:50]), weights)  # Usando um subset para demonstração
    if step % 10 == 0:
        c = cost(weights, X_train[:50, :n_qubits], y_train[:50])
        print(f"Step {step}: Cost = {c:.4f}")

# Avaliação
test_cost = cost(weights, X_test[:50, :n_qubits], y_test[:50])
print(f"Custo no conjunto de teste: {test_cost:.4f}")


In [ ]:
print("Distribuição dos rótulos de treinamento:", np.unique(y_train[:50], return_counts=True))
print("Distribuição dos rótulos de teste:", np.unique(y_test[:50], return_counts=True))


O resultado confirma que o conjunto de treinamento contém apenas a classe \( 0 \), e a classe \( 1 \) está ausente. Isso é um problema, pois o modelo não pode aprender a diferenciar entre as classes se apenas uma delas estiver presente.

### Soluções Possíveis
1. **Verificar o Dataset Original**:
   - Certifique-se de que o dataset original contém amostras de todas as classes e que os dados foram carregados corretamente.

2. **Balancear o Dataset**:
   - Se o dataset original for desequilibrado, tente aumentar ou incluir amostras da classe \( 1 \) no conjunto de treinamento.

3. **Ajustar a Divisão dos Dados**:
   - Reavalie a divisão de dados em treinamento e teste para garantir que ambos contenham todas as classes.

4. **Código para Checar o Dataset Original**:
   Para verificar a distribuição das classes no dataset completo:
   ```python
   print("Distribuição das classes no conjunto completo:", np.unique(y_train + y_test, return_counts=True))
   ```

5. **Exemplo de Balanceamento Manual**:
   Caso o dataset original tenha classes suficientes, você pode aumentar a classe minoritária:
   ```python
   # Reamostrando manualmente
   if len(class_1_indices) > 0:
       min_class_size = min(len(class_0_indices), len(class_1_indices))
       balanced_indices = np.hstack((
           resample(class_0_indices, n_samples=min_class_size, random_state=42),
           resample(class_1_indices, n_samples=min_class_size, random_state=42)
       ))

       X_train_balanced = X_train[balanced_indices]
       y_train_balanced = y_train[balanced_indices]

       print("Distribuição balanceada:", np.unique(y_train_balanced, return_counts=True))
   else:
       print("A classe 1 não está presente no conjunto original.")
   ```

Se o problema persistir, você pode me informar sobre os detalhes do dataset original para que possamos ajustar o pipeline de pré-processamento.

In [ ]:
# Verificar as classes presentes no conjunto de treinamento
print("Distribuição original dos rótulos:", np.unique(y_train, return_counts=True))

# Se não houver dados para uma classe, adicione ou corrija a amostragem
if len(class_1_indices) == 0:
    print("A classe 1 não está presente no conjunto de treinamento.")
else:
    # Balancear os dados se ambas as classes estiverem presentes
    min_class_size = min(len(class_0_indices), len(class_1_indices))
    balanced_indices = np.hstack((
        resample(class_0_indices, n_samples=min_class_size, random_state=42),
        resample(class_1_indices, n_samples=min_class_size, random_state=42)
    ))

    # Balancear X_train e y_train
    X_train_balanced = X_train[balanced_indices]
    y_train_balanced = y_train[balanced_indices]

    print("Distribuição balanceada dos rótulos de treinamento:", np.unique(y_train_balanced, return_counts=True))


In [ ]:
# Reamostrando manualmente
if len(class_1_indices) > 0:
    min_class_size = min(len(class_0_indices), len(class_1_indices))
    balanced_indices = np.hstack((
        resample(class_0_indices, n_samples=min_class_size, random_state=42),
        resample(class_1_indices, n_samples=min_class_size, random_state=42)
    ))

    X_train_balanced = X_train[balanced_indices]
    y_train_balanced = y_train[balanced_indices]

    print("Distribuição balanceada:", np.unique(y_train_balanced, return_counts=True))
else:
    print("A classe 1 não está presente no conjunto original.")


O resultado indica que a classe \( 1 \) está completamente ausente no dataset, não apenas no conjunto de treinamento, mas aparentemente no conjunto original carregado. Isso pode ser causado por:

1. **Problemas no Dataset Original**:
   - O dataset fornecido contém apenas amostras da classe \( 0 \).
2. **Erro na Organização do Dataset**:
   - Pode haver uma falha no pré-processamento ou na separação das classes.

### Próximos Passos

#### 1. Verificar o Dataset Original
Certifique-se de que o dataset original contém amostras de ambas as classes. Se houver subdiretórios como "maligno" e "benigno", confirme que ambos foram processados. Use o seguinte código para listar os diretórios e a contagem de arquivos em cada um:

```python
import os

# Listar subdiretórios no dataset
for root, dirs, files in os.walk(extract_path):
    print(f"Diretório: {root}, Número de arquivos: {len(files)}")
```

#### 2. Corrigir o Pipeline de Pré-processamento
Caso o problema esteja na seleção dos dados, ajuste o pipeline para incluir todas as classes. Certifique-se de que o código está capturando ambas as classes:

```python
class_dirs = [os.path.join(extract_path, "benigno"), os.path.join(extract_path, "maligno")]

# Processar imagens de todas as classes
all_image_paths = [img for class_dir in class_dirs for img in glob.glob(f"{class_dir}/*.jpg")]
processed_images, labels = process_images(all_image_paths, image_size)

# Verificar distribuição
print("Distribuição das classes no dataset completo:", np.unique(labels, return_counts=True))
```

#### 3. Adicionar Amostras da Classe Minoritária
Se o dataset original é desequilibrado, você pode adicionar manualmente mais amostras da classe \( 1 \) (maligno). Isso pode ser feito coletando mais dados ou duplicando algumas amostras existentes para balancear.

Se precisar de ajuda para investigar o dataset ou ajustar o pipeline, avise!

In [ ]:
import os

# Listar subdiretórios no dataset
for root, dirs, files in os.walk(extract_path):
    print(f"Diretório: {root}, Número de arquivos: {len(files)}")


In [ ]:
class_dirs = [os.path.join(extract_path, "benigno"), os.path.join(extract_path, "maligno")]

# Processar imagens de todas as classes
all_image_paths = [img for class_dir in class_dirs for img in glob.glob(f"{class_dir}/*.jpg")]
processed_images, labels = process_images(all_image_paths, image_size)

# Verificar distribuição
print("Distribuição das classes no dataset completo:", np.unique(labels, return_counts=True))


Agora temos uma distribuição equilibrada no dataset completo:

- **500 amostras da classe "benigno"**.
- **500 amostras da classe "maligno"**.

Isso confirma que ambas as classes estão presentes e foram corretamente processadas. O problema anterior foi causado por um desequilíbrio ou erro na seleção dos dados para treinamento.

### Próximos Passos

1. **Codificar os Rótulos**:
   - Converter os rótulos `['benigno', 'maligno']` para valores numéricos \(0\) e \(1\).

   ```python
   from sklearn.preprocessing import LabelEncoder

   # Codificar os rótulos
   label_encoder = LabelEncoder()
   encoded_labels = label_encoder.fit_transform(labels)
   print("Classes codificadas:", np.unique(encoded_labels, return_counts=True))
   ```

2. **Dividir o Dataset**:
   - Separar os dados em conjuntos de treinamento e teste de forma balanceada.
   
   ```python
   from sklearn.model_selection import train_test_split

   # Dividir os dados
   X_train, X_test, y_train, y_test = train_test_split(
       processed_images.reshape(len(processed_images), -1),  # Flatten as imagens
       encoded_labels,
       test_size=0.2,
       stratify=encoded_labels,  # Garantir balanceamento
       random_state=42
   )
   print("Distribuição dos rótulos no treinamento:", np.unique(y_train, return_counts=True))
   print("Distribuição dos rótulos no teste:", np.unique(y_test, return_counts=True))
   ```

3. **Treinar o Modelo Quântico**:
   - Use os dados balanceados para treinar o modelo quântico e avaliar o desempenho.

Deseja que eu implemente essas etapas ou passe direto para o treinamento?

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Codificar os rótulos
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
print("Classes codificadas:", np.unique(encoded_labels, return_counts=True))


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(
    processed_images.reshape(len(processed_images), -1),  # Flatten as imagens
    encoded_labels,
    test_size=0.2,
    stratify=encoded_labels,  # Garantir balanceamento
    random_state=42
)
print("Distribuição dos rótulos no treinamento:", np.unique(y_train, return_counts=True))
print("Distribuição dos rótulos no teste:", np.unique(y_test, return_counts=True))


Agora o dataset está devidamente balanceado:

- **Treinamento**: 400 amostras de cada classe (\( 0 \) e \( 1 \)).
- **Teste**: 100 amostras de cada classe (\( 0 \) e \( 1 \)).

Com os dados preparados, podemos seguir para o treinamento do modelo quântico.

### Próximos Passos

1. **Treinar o Modelo Quântico**:
   - Ajustar os pesos do circuito para minimizar a função de custo.
   - Utilizar o conjunto de treinamento balanceado (\( X\_train \) e \( y\_train \)).

2. **Avaliar o Modelo**:
   - Calcular a precisão e a perda no conjunto de teste (\( X\_test \) e \( y\_test \)).

### Código para Treinamento
Aqui está o código atualizado para treinar e avaliar o modelo:

```python
from pennylane.optimize import AdamOptimizer

# Função de custo
def cost(weights, X, y):
    loss = 0
    for i in range(len(X)):
        pred = circuit(weights, X[i])
        loss += (pred - y[i])**2
    return loss / len(X)

# Configurando o otimizador
opt = AdamOptimizer(stepsize=0.01)
steps = 50  # Número de iterações
weights = np.random.random(weights_shape)  # Reinicializando pesos

# Treinamento
for step in range(steps):
    weights = opt.step(lambda w: cost(w, X_train[:, :n_qubits], y_train), weights)
    if step % 10 == 0:
        c = cost(weights, X_train[:, :n_qubits], y_train)
        print(f"Step {step}: Cost = {c:.4f}")

# Avaliação no conjunto de teste
test_cost = cost(weights, X_test[:, :n_qubits], y_test)
print(f"Custo no conjunto de teste: {test_cost:.4f}")
```

### Explicação
1. **Função de Custo**:
   - Calcula o erro quadrático médio entre a previsão do circuito e os rótulos reais.
2. **Otimização**:
   - Usa Adam para ajustar os pesos do circuito.
3. **Avaliação**:
   - Mede o custo no conjunto de teste para verificar a generalização.

Deseja executar este código ou ajustar algum parâmetro antes de seguir?

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
from pennylane.optimize import AdamOptimizer

# Função de custo
def cost(weights, X, y):
    loss = 0
    for i in range(len(X)):
        pred = circuit(weights, X[i])
        loss += (pred - y[i])**2
    return loss / len(X)

# Configurando o otimizador
opt = AdamOptimizer(stepsize=0.01)
steps = 50  # Número de iterações
weights = np.random.random(weights_shape)  # Reinicializando pesos
train_costs = []
test_costs = []

# Configurando o gráfico
plt.ion()  # Ativando o modo interativo
fig, ax = plt.subplots()
line1, = ax.plot([], [], label='Custo de Treinamento', color='blue')
line2, = ax.plot([], [], label='Custo de Teste', color='orange')
ax.set_xlim(0, steps)
ax.set_ylim(0, 1)
ax.set_xlabel("Passo")
ax.set_ylabel("Custo")
ax.legend()
plt.show()

# Treinamento
start_time = time.time()
for step in range(steps):
    weights = opt.step(lambda w: cost(w, X_train[:, :n_qubits], y_train), weights)
    train_cost = cost(weights, X_train[:, :n_qubits], y_train)
    test_cost = cost(weights, X_test[:, :n_qubits], y_test)

    # Armazenar custos
    train_costs.append(train_cost)
    test_costs.append(test_cost)

    # Atualizar gráfico
    line1.set_data(range(step + 1), train_costs)
    line2.set_data(range(step + 1), test_costs)
    ax.set_ylim(0, max(train_costs + test_costs) * 1.1)  # Ajustar limites do gráfico dinamicamente
    clear_output(wait=True)
    plt.draw()
    plt.pause(0.1)

    # Print no console
    print(f"Passo {step}/{steps} | Custo de Treinamento: {train_cost:.4f} | Custo de Teste: {test_cost:.4f}")

end_time = time.time()
print(f"Treinamento concluído em {end_time - start_time:.2f} segundos.")
plt.ioff()  # Desativar o modo interativo
plt.show()

# Avaliação Final no Conjunto de Teste
final_test_cost = cost(weights, X_test[:, :n_qubits], y_test)
print(f"Custo final no conjunto de teste: {final_test_cost:.4f}")
